## IMPORTS

In [12]:
import pandas as pd
import numpy as np

## DATA EXPLORATION

In [5]:
df = pd.read_csv("Dirty_dataset.csv",sep=';',encoding='unicode_escape')
df

,Settore Merceologico,Insegna,Ubicazione,Tipo via,Via,Civico,Codice via,ZD,Settore Storico Cf Preval,Superficie Vendita,Superficie Altri Usi,Superficie Tabelle Speciali,Superficie Totale
0,NaN,NaN,CSO ITALIA N. 34 (z.d. 1),CSO,ITALIA,34,526,1.0,speciale panificatori,0.0,NaN,NaN,0.0
1,NaN,NaN,CSO LODI N. 104 (z.d. 4),CSO,LODI,104,4068,4.0,NaN,16.0,5.0,NaN,21.0
2,NaN,NaN,CSO LODI N. 26 (z.d. 5),CSO,LODI,26,4068,5.0,speciale panificatori,0.0,NaN,NaN,0.0
3,NaN,NaN,CSO SEMPIONE N. 41 (z.d. 6),CSO,SEMPIONE,41,7137,8.0,NaN,120.0,NaN,NaN,120.0
4,NaN,NaN,LGO SCALABRINI GIOVANNI BATTISTA N. 2 (z.d. 6),LGO,SCALABRINI GIOVANNI BATTISTA,2,5392,6.0,NaN,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24131,Tabella speciale Monopolio;Alimentare,NaN,RIP DI PTA TICINESE N. 61 ; isolato:071; acces...,RIP,DI PTA TICINESE,61,5150,6.0,cartoleria cancelleria,25.0,27.0,21.0,52.0
24132,Tabella speciale Monopolio;Alimentare;Non Alim...,i.p.,VIA GALVANI LUIGI N. 34 ; isolato:238; accesso...,VIA,GALVANI LUIGI,34,1179,2.0,carburanti e lubrificanti,45.0,NaN,NaN,60.0
24133,tabella speciale monopolio;non alimentare,tabacchi noli,VIA MARINO TOMMASO N. 7 ;gll vittorio emanuele...,VIA,MARINO TOMMASO,7,204,1.0,articoli per fumatori,28.0,12.0,NaN,40.0
24134,Tabella speciale Monopolio;Non Alimentare,NaN,VIA ASELLI GASPARE N. 11 (z.d. 3),VIA,ASELLI GASPARE,11,3167,3.0,commercio al dettaglio,7.0,125.0,5.0,132.0


In [6]:
df.shape

(24136, 13)

In [7]:
df.columns

Index(['Settore Merceologico', 'Insegna', 'Ubicazione', 'Tipo via', 'Via',
       'Civico', 'Codice via', 'ZD', 'Settore Storico Cf Preval',
       'Superficie Vendita', 'Superficie Altri Usi',
       'Superficie Tabelle Speciali', 'Superficie Totale'],
      dtype='object')

In [8]:
df.dtypes

Settore Merceologico            object
Insegna                         object
Ubicazione                      object
Tipo via                        object
Via                             object
Civico                          object
Codice via                      object
ZD                             float64
Settore Storico Cf Preval       object
Superficie Vendita             float64
Superficie Altri Usi           float64
Superficie Tabelle Speciali    float64
Superficie Totale              float64
dtype: object

## DQ ASSESMENT

The dimensions to evaluate are:

- completeness
- accuracy
- timeliness
- consistency

In [19]:
not_nulls = df.count().sum()
tot_cells = np.prod(df.shape)
completeness = not_nulls/tot_cells * 100
print(f"COMPLETENESS: {completeness :.2f}%")

COMPLETENESS: 82.12%


## DATA PROFILING

## DATA WRANGLING

## MISSING VALUES HANDLING

## OUTLIERS HANDLING

## DUPLICATES HANDLING

## PERFORMANCE ANALYSIS (optional)